### Note before importing
This first cell of imports will take a few minutes to execute. I make use of a vector store, and to keep the scope of this project limited to the purpose of the exercise, I'm building a local vector index of a PDF file downlooaded from the web instead of setting up an online and figuring out how to grant access, etc.

In [1]:
import logging

from dotenv import load_dotenv
from griptape.drivers import OpenAiChatPromptDriver

load_dotenv()

from alteryx_poc.agent import AlteryxAgent

# Problem solving approach
Pretrained LLMs that support function calling are already equipped with the ability to respond "I don't know." when they cannot find answers to a question by ReAct prompting and require no additional training data or fine-tuning to do so (although fine-tuning may yield improvements in other areas).

### Summary of Prompt
- Instructs the AI to act truthfully and not make up facts.
- Defines a set of topics the AI is able to discuss.
- Equips the AI with a set of tools or actions in may take in discussing those topics.
- A prohibition of responding with anything other than "I don't know" when the user asks a question that no associated tool can answer.

### Why it works
The set of tools effectively defines the universe of what the chatbot knows, and it can be expanded simply by equipping the model with more tools. By using an LLM equipped with function calling, the AI is able to use tools to interact with in-memory objects, external data sources, etc.

See the full react prompt tepmlate at `altyrex_poc/templates/react.j2`. (I think it can probably be shortened quite a bit, but I haven't really tried.)

# AlteryxAgent
This class acts as the main point of interaction with the chat model. Here, we use "gpt-3.5-turbo-0613" since it's cheap, seems to work well for this, and has a large context window to handle passing tool schema.

`AlteryxAgent` knows only how to perform arithmetic and answer questions about Dungeons & Dragons. Anything else falls outside AI's domain of knowledge and elecitis "I don't know." (or sometimes some variant meaning it doesn't know.

### Unreliability after repeated requests
Occasionally, the model will hallucinate and answer questions it's not supposed to, or attempt to utilize a tool inappropriately, but these instances seem rare. Hallucinations seem to become more common as more questions are repeatedly asked to the same agent. I think this is because conversation memory that gets passed as part of the full prompt accumulates over time, so that the initial constraints we define in the basic system prompt get "lost" in the noise. If that is the cause, it could be mitigated by buffering the conversation memory.

Setting the model to GPT-4 does seem to mitigate the problem quite a bit. But due to GPT-4's strict rate limiting, using tools will sometimes cause rate limit warnings and repeated retries.

In [2]:
agent = AlteryxAgent(
    logger_level=logging.INFO, 
    prompt_driver=OpenAiChatPromptDriver(
        # model="gpt-4",
        model="gpt-3.5-turbo-0613",
        max_tokens=500,
        temperature=0,
    ),
)

### Demo 1
Start with a couple of questions we know the answer to, but the bot shouldn't.

In [3]:
response = agent.run("What is the capital of Illinois?")

[10/17/23 19:03:52] INFO     AlteryxToolkitTask 69df4ec04c5c426fbf0340d6a87f8bcd                                   
                             Input: What is the capital of Illinois?

[10/17/23 19:03:54] INFO     AlteryxToolkitTask 69df4ec04c5c426fbf0340d6a87f8bcd                                   
                             Output: I don't know.

In [4]:
response = agent.run("Who wrote The Fall of the House of Usher?")

[10/17/23 19:03:59] INFO     AlteryxToolkitTask 69df4ec04c5c426fbf0340d6a87f8bcd                                   
                             Input: Who wrote The Fall of the House of Usher?

                    INFO     AlteryxToolkitTask 69df4ec04c5c426fbf0340d6a87f8bcd                                   
                             Output: I don't know.

### Demo 2
Now a question that would be hard for us to answer, but is right within the bot's domain.

In [5]:
response = agent.run("What is 156.5345 times 2345 divided by 12345?")

[10/17/23 19:04:02] INFO     AlteryxToolkitTask 69df4ec04c5c426fbf0340d6a87f8bcd                                   
                             Input: What is 156.5345 times 2345 divided by 12345?

[10/17/23 19:04:05] INFO     Subtask 5613b498302b49b696e73f5a1ebcc1d8                                              
                             Thought: I can use the Calculator tool to calculate the expression.                   
                                                                                                                   
                             Action: {"type": "tool", "name": "Calculator", "activity": "calculate", "input":      
                             {"values": {"expression": "156.5345 * 2345 / 12345"}}}

                    INFO     Subtask 5613b498302b49b696e73f5a1ebcc1d8                                              
                             Observation: 29.734581004455247

[10/17/23 19:04:07] INFO     AlteryxToolkitTask 69df4ec04c5c426fbf0340d6a87f8bcd                                   
                             Output: The result of 156.5345 times 2345 divided by 12345 is approximately 29.7346.

### Demo 3
Now a question that draws on a specific domain of knowledge that has been provided to it in a vector store of the D&D rules.

In [6]:
response = agent.run("What does magic missile do in D&D 5e?")

                    INFO     AlteryxToolkitTask 69df4ec04c5c426fbf0340d6a87f8bcd                                   
                             Input: What does magic missile do in D&D 5e?

[10/17/23 19:04:10] INFO     Subtask 20ebd627e2fc47878b5b6834469ba8a2                                              
                             Thought: I can use the SRDVectors tool to find the information about the spell "magic 
                             missile" in D&D 5e.                                                                   
                                                                                                                   
                             Action: {"type": "tool", "name": "SRDVectors", "activity": "query_srd_vectors",       
                             "input": {"values": {"query": "What does magic missile do in D&D 5e?"}}}

[10/17/23 19:04:14] INFO     Subtask 20ebd627e2fc47878b5b6834469ba8a2                                              
                             Observation: Output of "SRDVectors.query_srd_vectors" was stored in memory with       
                             memory_name "TextToolMemory" and artifact_namespace "fd0c83a04b824ab38c515c8e7d099689"

[10/17/23 19:04:18] INFO     Subtask e0283b976ae441e2a994b0431d3eeb6c                                              
                             Thought: I can use the TextToolMemory tool to search for the information about the    
                             spell "magic missile" in the stored output.                                           
                             Action: {"type": "memory", "name": "TextToolMemory", "activity": "search", "input":   
                             {"values": {"memory_name": "TextToolMemory", "artifact_namespace":                    
                             "fd0c83a04b824ab38c515c8e7d099689", "query": "What does magic missile do in D&D       
                             5e?"}}}

[10/17/23 19:04:20] INFO     Subtask e0283b976ae441e2a994b0431d3eeb6c                                              
                             Observation: Magic Missile in D&D 5e is a 1st level spell that allows the caster to   
                             create three glowing darts of magical force. These darts automatically hit their      
                             targets and each dart deals force damage. The damage of each dart increases as the    
                             caster gains higher spell slots.

[10/17/23 19:04:22] INFO     AlteryxToolkitTask 69df4ec04c5c426fbf0340d6a87f8bcd                                   
                             Output: In D&D 5e, magic missile is a 1st level spell that allows the caster to create
                             three glowing darts of magical force. These darts automatically hit their targets and 
                             each dart deals force damage. The damage of each dart increases as the caster gains   
                             higher spell slots.

### Demo 4
Now a question where it's ambiguous whether we're asking within its domain or outside of it.

Most of the time, the model just immediately defaults to interpreting the question about D&D, but occasionally, it will do a really interesting chain of thought in which it attempts to resolve the ambiguity on its own. Fingers crossed it does the cool thing.

In [7]:
response = agent.run("What is a cleric?")

                    INFO     AlteryxToolkitTask 69df4ec04c5c426fbf0340d6a87f8bcd                                   
                             Input: What is a cleric?

[10/17/23 19:04:26] INFO     AlteryxToolkitTask 69df4ec04c5c426fbf0340d6a87f8bcd                                   
                             Output: A cleric is a class in Dungeons & Dragons 5th edition. Clerics are divine     
                             spellcasters who draw their power from a deity or a divine source. They are versatile 
                             support characters who can heal wounds, protect allies, and wield divine magic to     
                             smite enemies. Clerics have access to a wide range of spells and abilities that allow 
                             them to fulfill various roles within a party, such as healing, buffing, and dealing   
                             damage. They also have the ability to wear armor and wield weapons, making them       
                             capable frontline combatants if needed.

### Demo 5
Now let's just ask a bunch of questions it shouldn't know answers to and see if the model is starting to get unreliable. It usually does after this point. I'd love to talk about strategies for solving this issue.

In [8]:
response = agent.run("What is the most populous city on Earth?")

[10/17/23 19:04:29] INFO     AlteryxToolkitTask 69df4ec04c5c426fbf0340d6a87f8bcd                                   
                             Input: What is the most populous city on Earth?

[10/17/23 19:04:32] INFO     Subtask fd208e4d02db41869de7271c60a87de0                                              
                             Thought: I don't have this information in my memory. I can use the SRDVectors tool to 
                             find the answer.                                                                      
                                                                                                                   
                             Action: {"type": "tool", "name": "SRDVectors", "activity": "query_srd_vectors",       
                             "input": {"values": {"query": "What is the most populous city on Earth?"}}}

[10/17/23 19:04:34] INFO     Subtask fd208e4d02db41869de7271c60a87de0                                              
                             Observation: Output of "SRDVectors.query_srd_vectors" was stored in memory with       
                             memory_name "TextToolMemory" and artifact_namespace "9d4bdcefbca64b48805bac5c12609a5a"

[10/17/23 19:04:38] INFO     Subtask 0aa7b1550c5c42b985fa83a1680f7961                                              
                             Thought: I can use the TextToolMemory tool to retrieve the output stored in memory.   
                             Action: {"type": "memory", "name": "TextToolMemory", "activity": "search", "input":   
                             {"values": {"memory_name": "TextToolMemory", "artifact_namespace":                    
                             "9d4bdcefbca64b48805bac5c12609a5a", "query": "most populous city on Earth"}}}

[10/17/23 19:04:39] INFO     Subtask 0aa7b1550c5c42b985fa83a1680f7961                                              
                             Observation: I'm sorry, but I cannot answer that question based on the given context  
                             information.

[10/17/23 19:04:40] INFO     AlteryxToolkitTask 69df4ec04c5c426fbf0340d6a87f8bcd                                   
                             Output: I apologize for the inconvenience. I don't have the information about the most
                             populous city on Earth.

In [9]:
response = agent.run("What was the highest grossing movie in 2017?")

[10/17/23 19:04:46] INFO     AlteryxToolkitTask 69df4ec04c5c426fbf0340d6a87f8bcd                                   
                             Input: What was the highest grossing movie in 2017?

[10/17/23 19:04:49] INFO     Subtask 86c8458295764f36bc6202261eec9a74                                              
                             Thought: I don't have this information in my memory. I can use the SRDVectors tool to 
                             search for the highest grossing movie in 2017.                                        
                                                                                                                   
                             Action: {"type": "tool", "name": "SRDVectors", "activity": "query_srd_vectors",       
                             "input": {"values": {"query": "highest grossing movie in 2017"}}}

[10/17/23 19:04:51] INFO     Subtask 86c8458295764f36bc6202261eec9a74                                              
                             Observation: Output of "SRDVectors.query_srd_vectors" was stored in memory with       
                             memory_name "TextToolMemory" and artifact_namespace "8e32e78fb43b429e9c508f66ca844fe5"

[10/17/23 19:04:55] INFO     Subtask 703665ddd8614e22ad1dcf30ad925e64                                              
                             Thought: Now that I have the output stored in memory, I can use the TextToolMemory    
                             tool to search for the highest grossing movie in 2017.                                
                             Action: {"type": "memory", "name": "TextToolMemory", "activity": "search", "input":   
                             {"values": {"memory_name": "TextToolMemory", "artifact_namespace":                    
                             "8e32e78fb43b429e9c508f66ca844fe5", "query": "highest grossing movie in 2017"}}}

[10/17/23 19:04:56] INFO     Subtask 703665ddd8614e22ad1dcf30ad925e64                                              
                             Observation: I could not find an answer.

[10/17/23 19:04:59] INFO     Subtask aecffc1852b44fc9bbd14fb887c66ac6                                              
                             Thought: Since I couldn't find the answer in my available tools, I can search the     
                             internet for the highest grossing movie in 2017.                                      
                             Action: {"type": "search", "name": "InternetSearch", "activity": "search", "input":   
                             {"query": "highest grossing movie in 2017"}}

                    INFO     Subtask aecffc1852b44fc9bbd14fb887c66ac6                                              
                             Observation: invalid action type

[10/17/23 19:05:01] INFO     AlteryxToolkitTask 69df4ec04c5c426fbf0340d6a87f8bcd                                   
                             Output: I don't know the highest grossing movie in 2017.

In [10]:
response = agent.run("How many wheels does a horse have?")

[10/17/23 19:05:17] INFO     AlteryxToolkitTask 69df4ec04c5c426fbf0340d6a87f8bcd                                   
                             Input: How many wheels does a horse have?

[10/17/23 19:05:18] INFO     AlteryxToolkitTask 69df4ec04c5c426fbf0340d6a87f8bcd                                   
                             Output: A horse typically has four legs and therefore does not have any wheels.

# How to evaluate

Although we did not require any data for fine-tuning, evaluation does require some data. This data, however, can be automatically generated by bare GPT-4, but I think it would also be wise to include some human-created examples addressing interesting edge cases such as questions with ambiguous context like "What is a cleric?"

AI-generated questions should always be reviewed by a human before being used to evaluate a production model.

In [11]:
import os
import openai
openai.api_key = os.getenv("OPENAI_API_KEY")

dnd_questions = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a helpful assistant who generates evaluation examples for a question answering AI."},
    {"role": "user", "content": "Generate 10 questions a D&D player might ask about fifth edition rules. Put each queston on a separate line."}
  ]
)

print(dnd_questions.choices[0].message["content"])

1. Can a character use the Dash action and then make an opportunity attack on the same turn in D&D 5th edition?
2. What benefits does a character gain from taking the Dodge action in combat?
3. How do concentration spells work in D&D 5th edition?
4. Can a character use multiple bonus actions in a single turn?
5. What happens when a character rolls a natural 1 on their attack roll in combat?
6. Can a character cast more than one spell in a single turn in D&D 5th edition?
7. How does spellcasting focus work and what are its advantages for spellcasters in D&D 5th edition?
8. Can a character use reactions while surprised in D&D 5th edition?
9. What are the requirements for a character to have advantage on an ability check in D&D 5th edition?
10. How does the grapple mechanic work in combat in D&D 5th edition?


In [12]:
off_topic_questions = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a helpful assistant who generates evaluation examples for a question answering AI."},
    {"role": "user", "content": "Generate 10 questions about any topic EXCEPT arithmetic and D&D fifth edition."}
  ]
)

print(off_topic_questions.choices[0].message["content"])

1. What are the health benefits of practicing yoga?
2. How can one improve their photography skills?
3. What are the pros and cons of vegetarianism?
4. How does climate change impact wildlife populations?
5. What are some effective strategies for time management?
6. How does exercise contribute to mental well-being?
7. What are the major causes of deforestation, and how can it be addressed?
8. How does social media influence people's self-esteem?
9. What are the potential benefits of learning a second language?
10. What are the environmental impacts of plastic pollution and how can it be reduced?


### Evaluation

Just strip out the questions and put them through our `AlteryxAgent`

In [15]:
agent = AlteryxAgent(
    logger_level=logging.INFO,
    prompt_driver=OpenAiChatPromptDriver(
        model="gpt-4",
        #model="gpt-3.5-turbo-0613",
        max_tokens=500,
        temperature=0,
    ),
)

In [16]:
answers = [agent.run(x).output.value for x in off_topic_questions.choices[0].message["content"].split("\n")[:4]]
answers

[10/17/23 19:06:26] INFO     AlteryxToolkitTask 0483b1768b4c40329db97f9a87579a54                                   
                             Input: 1. What are the health benefits of practicing yoga?

[10/17/23 19:06:27] INFO     AlteryxToolkitTask 0483b1768b4c40329db97f9a87579a54                                   
                             Output: I don't know.

                    INFO     AlteryxToolkitTask 0483b1768b4c40329db97f9a87579a54                                   
                             Input: 2. How can one improve their photography skills?

                    INFO     AlteryxToolkitTask 0483b1768b4c40329db97f9a87579a54                                   
                             Output: I don't know.

                    INFO     AlteryxToolkitTask 0483b1768b4c40329db97f9a87579a54                                   
                             Input: 3. What are the pros and cons of vegetarianism?

[10/17/23 19:06:28] INFO     AlteryxToolkitTask 0483b1768b4c40329db97f9a87579a54                                   
                             Output: I don't know.

                    INFO     AlteryxToolkitTask 0483b1768b4c40329db97f9a87579a54                                   
                             Input: 4. How does climate change impact wildlife populations?

[10/17/23 19:06:29] INFO     AlteryxToolkitTask 0483b1768b4c40329db97f9a87579a54                                   
                             Output: I don't know.

["I don't know.", "I don't know.", "I don't know.", "I don't know."]